In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import shutil
import tensorflow as tf
import pathlib
import PIL
import time
import zipfile
import random
from tensorflow import keras as ks
from tensorflow.keras.layers import *
from keras.utils.np_utils import to_categorical 
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [36]:
train_df=pd.read_csv("../input/trailblazers-south-african-qualification-challenge/Train (5).csv")
test_df=pd.read_csv("../input/trailblazers-south-african-qualification-challenge/Test (7).csv")
sub=pd.read_csv("../input/trailblazers-south-african-qualification-challenge/SampleSubmission (3).csv")

In [37]:
Id=test_df["Place_ID X Date"]

In [38]:
train_df.head()

,Place_ID X Date,Date,Place_ID,target,target_min,target_max,target_variance,target_count,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,...,L3_SO2_sensor_zenith_angle,L3_SO2_solar_azimuth_angle,L3_SO2_solar_zenith_angle,L3_CH4_CH4_column_volume_mixing_ratio_dry_air,L3_CH4_aerosol_height,L3_CH4_aerosol_optical_depth,L3_CH4_sensor_azimuth_angle,L3_CH4_sensor_zenith_angle,L3_CH4_solar_azimuth_angle,L3_CH4_solar_zenith_angle
0,010Q650 X 2020-01-02,2020-01-02,010Q650,38.0,23.0,53.0,769.50,92,11.000000,60.200001,...,38.593017,-61.752587,22.363665,1793.793579,3227.855469,0.010579,74.481049,37.501499,-62.142639,22.545118
1,010Q650 X 2020-01-03,2020-01-03,010Q650,39.0,25.0,63.0,1319.85,91,14.600000,48.799999,...,59.624912,-67.693509,28.614804,1789.960449,3384.226562,0.015104,75.630043,55.657486,-53.868134,19.293652
2,010Q650 X 2020-01-04,2020-01-04,010Q650,24.0,8.0,56.0,1181.96,96,16.400000,33.400002,...,49.839714,-78.342701,34.296977,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,010Q650 X 2020-01-05,2020-01-05,010Q650,49.0,10.0,55.0,1113.67,96,6.911948,21.300001,...,29.181258,-73.896588,30.545446,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,010Q650 X 2020-01-06,2020-01-06,010Q650,21.0,9.0,52.0,1164.82,95,13.900001,44.700001,...,0.797294,-68.612480,26.899694,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
L=[i for i in train_df.columns if i not in test_df.columns]
L

['target', 'target_min', 'target_max', 'target_variance', 'target_count']

In [40]:
train_df["target"].nunique()

375

In [41]:
train_df.isna().sum().sort_values(ascending =False).head(20)

L3_CH4_solar_zenith_angle                              24765
L3_CH4_solar_azimuth_angle                             24765
L3_CH4_sensor_zenith_angle                             24765
L3_CH4_sensor_azimuth_angle                            24765
L3_CH4_aerosol_optical_depth                           24765
L3_CH4_aerosol_height                                  24765
L3_CH4_CH4_column_volume_mixing_ratio_dry_air          24765
L3_NO2_tropospheric_NO2_column_number_density           8669
L3_HCHO_tropospheric_HCHO_column_number_density_amf     7425
L3_HCHO_tropospheric_HCHO_column_number_density         7425
L3_HCHO_solar_zenith_angle                              7425
L3_HCHO_solar_azimuth_angle                             7425
L3_HCHO_sensor_zenith_angle                             7425
L3_HCHO_sensor_azimuth_angle                            7425
L3_HCHO_cloud_fraction                                  7425
L3_HCHO_HCHO_slant_column_number_density                7425
L3_SO2_absorbing_aerosol

In [42]:
train_df.isna().sum().sort_values(ascending =False).head(20)/len(train_df)

L3_CH4_solar_zenith_angle                              0.810453
L3_CH4_solar_azimuth_angle                             0.810453
L3_CH4_sensor_zenith_angle                             0.810453
L3_CH4_sensor_azimuth_angle                            0.810453
L3_CH4_aerosol_optical_depth                           0.810453
L3_CH4_aerosol_height                                  0.810453
L3_CH4_CH4_column_volume_mixing_ratio_dry_air          0.810453
L3_NO2_tropospheric_NO2_column_number_density          0.283699
L3_HCHO_tropospheric_HCHO_column_number_density_amf    0.242989
L3_HCHO_tropospheric_HCHO_column_number_density        0.242989
L3_HCHO_solar_zenith_angle                             0.242989
L3_HCHO_solar_azimuth_angle                            0.242989
L3_HCHO_sensor_zenith_angle                            0.242989
L3_HCHO_sensor_azimuth_angle                           0.242989
L3_HCHO_cloud_fraction                                 0.242989
L3_HCHO_HCHO_slant_column_number_density

In [43]:
train_df.drop(["L3_CH4_solar_zenith_angle","L3_CH4_solar_azimuth_angle","L3_CH4_sensor_zenith_angle","L3_CH4_sensor_azimuth_angle","L3_CH4_aerosol_optical_depth",
             "L3_CH4_aerosol_height","L3_CH4_CH4_column_volume_mixing_ratio_dry_air","Place_ID X Date","Place_ID"], axis=1, inplace=True)

In [44]:
train_df.shape

(30557, 73)

In [45]:
import datetime as dt
train_df["Date"]=pd.to_datetime(train_df["Date"])
train_df['Date_year'] =train_df["Date"].dt.year
train_df['Date_month'] = train_df["Date"].dt.month
train_df['Date_day'] = train_df["Date"].dt.day
train_df.drop("Date", axis=1, inplace=True)

In [46]:
df_train=train_df.drop("target",axis=1)

In [47]:
test_df.drop(["L3_CH4_solar_zenith_angle","L3_CH4_solar_azimuth_angle","L3_CH4_sensor_zenith_angle","L3_CH4_sensor_azimuth_angle","L3_CH4_aerosol_optical_depth",
             "L3_CH4_aerosol_height","L3_CH4_CH4_column_volume_mixing_ratio_dry_air","Place_ID X Date","Place_ID"], axis=1, inplace=True)

test_df["Date"]=pd.to_datetime(test_df["Date"])
test_df['Date_year'] =test_df["Date"].dt.year
test_df['Date_month'] = test_df["Date"].dt.month
test_df['Date_day'] = test_df["Date"].dt.day
test_df.drop("Date", axis=1, inplace=True)

In [48]:
df_test=test_df

In [49]:
y_train = train_df[["target"]]

In [50]:
df_train.isna().sum()

target_min                                 0
target_max                                 0
target_variance                            0
target_count                               0
precipitable_water_entire_atmosphere       0
                                        ... 
L3_SO2_solar_azimuth_angle              7237
L3_SO2_solar_zenith_angle               7237
Date_year                                  0
Date_month                                 0
Date_day                                   0
Length: 74, dtype: int64

In [51]:
import random
missing_values = df_train.columns[df_train.isna().any()].to_list()
for each in missing_values:
    if (df_train[each].dtypes =='float64'):
        minimum= int(df_train[each].quantile(0.25))
        maximum= int(df_train[each].quantile(0.75))
        A=df_train[df_train[each].isnull()].index.tolist()
        for i in A:
            df_train.loc[i,each]=random.randint(minimum,maximum)
        df_train[each]=pd.to_numeric(df_train[each])
   

    elif(df_train[each].dtypes == 'object'):
        if ('True' in str(df_train[each].str.contains('No').unique().tolist())):
            df_train[each].fillna('No',inplace=True)
        elif('True' in str(df_train[each].str.contains('None').unique().tolist())):
            df_train[each].fillna('None',inplace=True)
        elif('True' in str(df_train[each].str.contains('Unf').unique().tolist())):
            df_train[each].fillna('Unf',inplace=True)
        else:
            A=df_train[df_train[each].isnull()].index.tolist()
            unique = df_train[each].unique().tolist()
            unique=pd.Series(unique).dropna().tolist()
            for i in A:
                df_train.loc[i,each]=random.choice(unique)


missing_values = df_test.columns[df_test.isna().any()].to_list()
for each in missing_values:
    if (df_test[each].dtypes =='float64'):
        minimum= int(df_test[each].quantile(0.25))
        maximum= int(df_test[each].quantile(0.75))
        A=df_test[df_test[each].isnull()].index.tolist()
        for i in A:
            df_test.loc[i,each]=random.randint(minimum,maximum)
        df_test[each]=pd.to_numeric(df_test[each])
   

    elif(df_test[each].dtypes == 'object'):
        if ('True' in str(df_test[each].str.contains('No').unique().tolist())):
            df_test[each].fillna('No',inplace=True)
        elif('True' in str(df_test[each].str.contains('None').unique().tolist())):
            df_test[each].fillna('None',inplace=True)
        elif('True' in str(df_test[each].str.contains('Unf').unique().tolist())):
            df_test[each].fillna('Unf',inplace=True)
        else:
            A=df_test[df_test[each].isnull()].index.tolist()
            unique = df_test[each].unique().tolist()
            unique=pd.Series(unique).dropna().tolist()
            for i in A:
                df_test.loc[i,each]=random.choice(unique)

In [52]:
df_train.drop(['target_min', 'target_max', 'target_variance', 'target_count'], axis=1, inplace=True)

In [53]:
df_train.shape, df_test.shape

((30557, 70), (16136, 70))

In [54]:
df_train.describe()

,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,L3_NO2_NO2_column_number_density,L3_NO2_NO2_slant_column_number_density,L3_NO2_absorbing_aerosol_index,L3_NO2_cloud_fraction,...,L3_SO2_SO2_slant_column_number_density,L3_SO2_absorbing_aerosol_index,L3_SO2_cloud_fraction,L3_SO2_sensor_azimuth_angle,L3_SO2_sensor_zenith_angle,L3_SO2_solar_azimuth_angle,L3_SO2_solar_zenith_angle,Date_year,Date_month,Date_day
count,30557.000000,30557.000000,30557.000000,30557.000000,30557.000000,30557.000000,30557.000000,30557.000000,30557.000000,30557.000000,...,30557.000000,30557.000000,30557.000000,30557.000000,30557.000000,30557.000000,30557.000000,30557.0,30557.000000,30557.000000
mean,15.302326,70.552747,0.006004,9.321342,0.416886,0.088486,0.000087,0.000152,-1.051840,0.258096,...,0.000024,-1.839157,0.163228,-13.729229,35.542532,-127.876237,46.525337,2020.0,2.095755,15.254312
std,10.688573,18.807884,0.003787,9.343226,2.707990,2.641924,0.000094,0.000113,0.622454,0.299712,...,0.000147,0.690211,0.165263,70.395306,17.242990,63.632596,13.117769,0.0,0.899081,8.921199
min,0.420044,5.128572,0.000139,-34.647879,-15.559646,-15.023647,-0.000051,0.000000,-3.941742,0.000000,...,-0.004309,-4.833706,0.000000,-111.997123,0.000000,-179.880630,0.000000,2020.0,1.000000,1.000000
25%,7.666667,58.600002,0.003403,3.123071,-1.097864,-1.415701,0.000050,0.000113,-1.487721,0.014184,...,-0.000020,-2.233243,0.000000,-95.120394,22.906707,-163.609412,38.424371,2020.0,1.000000,7.000000
50%,12.200000,74.099998,0.004912,8.478424,0.222092,0.012847,0.000072,0.000146,-1.110044,0.118137,...,0.000000,-1.965179,0.114964,-14.235882,37.000000,-152.660601,47.000000,2020.0,2.000000,15.000000
75%,19.900000,85.450001,0.007562,16.201563,1.772925,1.555559,0.000107,0.000185,-0.700866,0.435056,...,0.000071,-1.372949,0.295409,67.459420,49.803167,-127.000000,55.288862,2020.0,3.000000,23.000000
max,72.599998,100.000000,0.021615,37.437921,17.955124,15.055530,0.002998,0.002431,4.824802,1.000000,...,0.006988,3.736877,0.618176,80.544067,66.111289,179.776125,79.631711,2020.0,4.000000,31.000000


In [55]:
col_tr=[i for i in df_train.columns]
col_ts=[i for i in df_test.columns]


In [56]:
df_train["mean"]=df_train.mean(axis=1)
df_train["std"]=df_train.std(axis=1)
df_train["min"]=df_train.min(axis=1)
df_train["max"]=df_train.max(axis=1)
df_train["kurt"]=df_train.kurt(axis=1)
df_train["skew"]=df_train.skew(axis=1)
df_train["q1"]=df_train.quantile(q=0.1,axis=1)
df_train["q2"]=df_train.quantile(q=0.2,axis=1)
df_train["q3"]=df_train.quantile(q=0.3,axis=1)
df_train["q4"]=df_train.quantile(q=0.4,axis=1)
df_train["q5"]=df_train.quantile(q=0.5,axis=1)
df_train["q6"]=df_train.quantile(q=0.6,axis=1)
df_train["q7"]=df_train.quantile(q=0.7,axis=1)
df_train["q8"]=df_train.quantile(q=0.8,axis=1)
df_train["q9"]=df_train.quantile(q=0.9,axis=1)
df_train.drop(columns=col_tr)

,mean,std,min,max,kurt,skew,q1,q2,q3,q4,q5,q6,q7,q8,q9
0,38327.075509,170128.059762,-61.789016,840209.874619,14.088649,3.967519,-31.799097,-0.000008,0.000058,0.157780,12.544325,22.363665,38.630734,76.537078,5554.251953
1,39020.495039,170481.655813,-74.457583,841116.763051,13.983037,3.948776,-62.354355,-12.056951,0.000057,0.019091,0.374680,13.983037,28.614804,59.624912,4915.251953
2,38984.645992,170554.414030,-105.201338,841410.713456,13.994883,3.950839,-90.924764,-62.988751,0.000060,0.032954,0.633730,13.994883,34.296977,49.839714,5724.308398
3,39026.381817,170482.030056,-104.334066,841103.242368,13.995072,3.950847,-87.935251,-59.365121,0.000081,0.068865,0.792261,6.911948,29.181258,30.545446,8449.771833
4,37998.689876,170126.515002,-68.622851,840763.055499,14.145970,3.977823,-34.306235,0.000205,0.037919,0.456309,0.898647,13.900001,26.899694,58.850179,9438.105953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30552,38903.243687,168976.672003,-19.340150,833666.850330,13.981010,3.948287,-10.833738,0.000020,0.001120,0.180711,6.287689,12.312248,49.871571,54.347408,16057.651751
30553,38751.232505,168948.985483,-156.993340,833894.342749,13.993026,3.950442,-85.223618,-2.312784,0.000041,0.007644,0.491874,6.300000,41.077206,49.881356,12288.334189
30554,37271.705642,168560.966383,-162.861860,833315.148608,14.188379,3.985682,-133.262897,-77.694965,0.000068,0.025447,0.189884,8.955216,13.065002,48.462347,16104.360312
30555,38616.554542,168870.637870,-168.907480,833509.685202,14.021247,3.955397,-85.839556,0.000019,0.045210,0.264140,12.297242,18.000000,47.293175,71.096345,12288.782564


In [57]:
df_test["mean"]=df_test.mean(axis=1)
df_test["std"]=df_test.std(axis=1)
df_test["min"]=df_test.min(axis=1)
df_test["max"]=df_test.max(axis=1)
df_test["kurt"]=df_test.kurt(axis=1)
df_test["skew"]=df_test.skew(axis=1)
df_test["q1"]=df_test.quantile(q=0.1,axis=1)
df_test["q2"]=df_test.quantile(q=0.2,axis=1)
df_test["q3"]=df_test.quantile(q=0.3,axis=1)
df_test["q4"]=df_test.quantile(q=0.4,axis=1)
df_test["q5"]=df_test.quantile(q=0.5,axis=1)
df_test["q6"]=df_test.quantile(q=0.6,axis=1)
df_test["q7"]=df_test.quantile(q=0.7,axis=1)
df_test["q8"]=df_test.quantile(q=0.8,axis=1)
df_test["q9"]=df_test.quantile(q=0.9,axis=1)
df_test.drop(columns=col_ts)

,mean,std,min,max,kurt,skew,q1,q2,q3,q4,q5,q6,q7,q8,q9
0,38137.432020,169206.737199,-95.987015,835670.492740,14.088822,3.967470,-48.062721,0.000123,0.032071,0.726971,1.445658,11.600000,22.942019,68.099367,5087.193183
1,37540.573110,168989.240881,-95.017976,835281.882757,14.167694,3.981837,-47.928810,0.000046,0.029557,0.665311,11.238063,18.539524,34.641758,75.936844,7732.436613
2,37796.237853,168943.994217,-94.015418,834839.050781,14.136103,3.976023,-47.359103,0.000045,0.007113,0.734215,10.275780,14.143972,55.872276,75.552445,6048.574601
3,38181.500007,169327.180048,-102.285125,836269.833912,14.086840,3.967114,-98.950745,-77.944103,0.000016,0.026926,0.527917,14.086840,32.730747,59.174917,5097.985842
4,38150.750559,169287.555947,-102.133960,836043.111009,14.086939,3.967134,-98.156094,-76.901677,0.000056,0.042078,0.569013,6.874006,28.320528,40.925873,5080.639945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16131,26103.482634,138988.185156,-176.521720,833357.423137,20.486105,4.686315,-36.500000,0.000000,0.000000,0.000826,0.998320,18.000000,38.857262,62.574317,2743.117227
16132,26051.757783,138948.287442,-17.869847,833249.919439,20.496714,4.687904,-1.660348,0.000000,0.000005,0.187120,1.002122,15.080328,40.751344,55.223046,2313.779789
16133,26422.302725,139053.540332,-152.298838,832603.364743,20.373239,4.669613,-99.663430,-1.503023,0.000000,0.000004,0.302597,2.000000,27.074135,41.197583,1580.513201
16134,14710.175498,99793.484062,-158.883352,832687.223024,32.881362,5.818099,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.171267,7.320778,229.738597


In [58]:
df_test.shape, df_train.shape 

((16136, 85), (30557, 85))

In [59]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
X_train = mms.fit_transform(df_train)
X_test = mms.transform(df_test)

In [60]:
y_train = mms.fit_transform(y_train)

In [61]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X_train, y_train, test_size=0.1)

In [62]:
# from sklearn.decomposition import PCA
# pca = PCA()
# rd_xtrain = pca.fit_transform(xtrain)
# rd_xtest = pca.transform(xtest)

In [63]:
# x_test=pca.fit_transform(X_test)[:,:15]

In [64]:
# explained_variance=pca.explained_variance_ratio_
# explained_variance.shape

In [65]:
# with plt.style.context('dark_background'):
#     plt.figure(figsize=(12, 6))

#     plt.bar(range(85), explained_variance, alpha=0.6, align='center',label='individual explained variance')
#     plt.ylabel('Explained variance ratio')
#     plt.xlabel('Principal components')
#     plt.legend(loc='best')
#     plt.tight_layout()

In [66]:
# xtrain_ = rd_xtrain[:, :15]
# xtest_ = rd_xtest[:, :15]

In [67]:
# xtrain_.shape, xtest_.shape

In [68]:
from sklearn.ensemble import RandomForestRegressor
Rf = RandomForestRegressor(min_samples_split=15)
Rf.fit(xtrain, ytrain)
ypred=Rf.predict(xtest)

In [69]:
from sklearn.metrics import mean_squared_error
mean_squared_error(ytest, ypred)

0.0015325193876962809

In [83]:
Rf.fit(xtest, ytest)

RandomForestRegressor(max_features=1, max_leaf_nodes=8, min_samples_leaf=10)

In [84]:
from sklearn.metrics import mean_squared_error
mean_squared_error(ytest, ypred)

0.0015325193876962809

In [ ]:
y_pred4 = Rf.predict(X_test) 
y_pred_original4 = mms.inverse_transform(y_pred4.reshape(-1,1))


In [ ]:
y_pred_original4=y_pred_original4.tolist()
y_pred_original4=[i[0] for i in y_pred_original4]
submission = pd.DataFrame({'Place_ID X Date	': Id,'target': y_pred_original4})
submission.to_csv('submission20.csv', index=False)

****

In [85]:
from sklearn.ensemble import RandomForestRegressor
Rf = RandomForestRegressor(min_samples_split=2, min_samples_leaf=10, max_features=1, max_leaf_nodes=8)
Rf.fit(xtrain, ytrain)
yprede=Rf.predict(xtest)
mean_squared_error(ytest, yprede)

0.002952483241755346

****

In [86]:
from sklearn.linear_model import Lasso
Ls=Lasso()
Ls.fit(xtrain, ytrain)
ypredd=Ls.predict(xtest)
mean_squared_error(ytest, ypredd)

0.0034219808721609388

In [87]:
from sklearn.tree import DecisionTreeRegressor
mytre=DecisionTreeRegressor(max_depth=8)
mytre.fit(xtrain, ytrain)
ypreddd=mytre.predict(xtest)
mean_squared_error(ytest, ypreddd)

0.0022136340215340613

In [ ]:
mytre.fit(xtest, ytest)
y_pred4 = Rf.predict(X_test) 
y_pred_original4 = mms.inverse_transform(y_pred4.reshape(-1,1))
y_pred_original4=y_pred_original4.tolist()
y_pred_original4=[i[0] for i in y_pred_original4]
submission = pd.DataFrame({'Place_ID X Date	': Id,'target': y_pred_original4})
submission.to_csv('submission21.csv', index=False)


****

In [88]:
from sklearn.neighbors import KNeighborsRegressor
KNN=KNeighborsRegressor(n_neighbors=20)
KNN.fit(xtrain, ytrain)
ypredk=KNN.predict(xtest)
mean_squared_error(ytest, ypredk)

0.0025784362820938968

****

In [ ]:
from sklearn.linear_model import RANSACRegressor
pf=RANSACRegressor()
pf.fit(xtrain, ytrain)
ypredp=pf.predict(xtest)
mean_squared_error(ytest, ypredp)

In [89]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(xtrain, ytrain)
ypredp=lr.predict(xtest)
mean_squared_error(ytest, ypredp)

0.0023910185366046786

****

In [81]:
from sklearn.svm import SVR
sv =SVR()
sv.fit(xtrain, ytrain)
ypredv=sv.predict(xtest)
mean_squared_error(ytest, ypredv)

0.0034133281140456308

****

In [ ]:
from sklearn import  ensemble
params = {
    "n_estimators": 500,
    "max_depth": 8,
    "min_samples_split": 5,
    "learning_rate": 0.01,
    "loss": "squared_error",
}
reg = ensemble.GradientBoostingRegressor(**params)
reg.fit(xtrain, ytrain)
mse = mean_squared_error(ytest, reg.predict(xtest))
mse

In [ ]:
mse

In [ ]:
reg.fit(xtest, ytest)
y_pred4 = reg.predict(x_test) 
y_pred_original4 = mms.inverse_transform(y_pred4.reshape(-1,1))
y_pred_original4=y_pred_original4.tolist()
y_pred_original4=[i[0] for i in y_pred_original4]
submission = pd.DataFrame({'Place_ID X Date	': Id,'target': y_pred_original4})
submission.to_csv('submission11.csv', index=False)

****

In [ ]:
from xgboost import XGBRegressor
model = XGBRegressor(objective='reg:squarederror')
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, xtrain, ytrain, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1, error_score='raise')
print('MSE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

In [ ]:
model.fit(xtrain, ytrain)
yhat = model.predict(xtest_)

In [ ]:
mean_squared_error(ytest, yhat)

In [ ]:
model.fit(xtest, ytest)

In [ ]:
y_pred4 = model.predict(x_test) 
y_pred_original4 = mms.inverse_transform(y_pred4.reshape(-1,1))
y_pred_original4=y_pred_original4.tolist()
y_pred_original4=[i[0] for i in y_pred_original4]
submission = pd.DataFrame({'Place_ID X Date	': Id,'target': y_pred_original4})
submission.to_csv('submission12.csv', index=False)

****

In [ ]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
model1 = LGBMRegressor()
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model1, xtrain, ytrain, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1, error_score='raise')
print('MAE: %.5f (%.5f)' % (mean(n_scores), std(n_scores)))


In [ ]:
model1 = LGBMRegressor()
model1.fit(xtrain, ytrain)
yhatt = model1.predict(xtest)
mean_squared_error(ytest, yhatt)

In [ ]:
model1.fit(xtest, ytest)

In [ ]:
y_pred4 = model1.predict(x_test) 
y_pred_original4 = mms.inverse_transform(y_pred4.reshape(-1,1))
y_pred_original4=y_pred_original4.tolist()
y_pred_original4=[i[0] for i in y_pred_original4]
submission = pd.DataFrame({'Place_ID X Date	': Id,'target': y_pred_original4})
submission.to_csv('submission13.csv', index=False)

In [ ]:
mean_squared_error(ytest, yhat)